In [114]:
import pandas as pd
import numpy as np 

In [201]:
rosters = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\rosters.csv")
seasons = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\seasons.csv")
teams = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\teams.csv")
players = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\players.csv")
gids = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\game_ids.csv")
draft = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\draft.csv")
pwins = pd.read_csv("\\Users\\jwegne1\OneDrive - General Motors Financial Company, Inc\\Documents\\nba_prediction-id_gather\\03_data\\playoff_wins.csv")

In [202]:
gids = gids[(gids['TEAM_ID'].isin([1610612737,1610612744]))&(gids['SEASON_YEAR'].isin(['2020-21','2019-20']))]
gids = gids.sample(15,random_state=41)

In [203]:
headers = {'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [204]:
import pandas as pd
import time
from nba_api.stats.endpoints import boxscorefourfactorsv2

def hit_boxscorefourfactorsv2(gs):
    # takes the game id data as input 
    # returns the four factors data for each player in each game
    bsffs = []
    for i in gs['GAME_ID']:
        bsff = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id=i[1:],headers = headers).get_data_frames()[0]
        bsffs.append(bsff)
        print(i)
        time.sleep(.6)
    BSFF = pd.concat(bsffs)
    BSFF['GAME_ID'] = 'G'+BSFF['GAME_ID']
    return BSFF    

In [205]:
bs = hit_boxscorefourfactorsv2(gids)

G0021900502
G0021900930
G0022000047
G0021900815
G0021900588
G0022000824
G0022000978
G0021900586
G0022000733
G0022000958
G0022000861
G0022000626
G0021900457
G0021900878
G0022000770


In [206]:
bs

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT
0,G0021900502,1610612744,GSW,Golden State,203922,Glenn Robinson III,Glenn,F,,39.000000:33,0.581,0.206,0.117,0.171,0.554,0.176,0.076,0.206
1,G0021900502,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,F,,35.000000:48,0.590,0.180,0.133,0.212,0.546,0.169,0.096,0.138
2,G0021900502,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,C,,33.000000:57,0.552,0.207,0.111,0.219,0.532,0.113,0.091,0.172
3,G0021900502,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,G,,39.000000:33,0.536,0.214,0.115,0.179,0.521,0.181,0.065,0.176
4,G0021900502,1610612744,GSW,Golden State,202692,Alec Burks,Alec,G,,38.000000:14,0.551,0.191,0.084,0.222,0.544,0.162,0.105,0.161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,G0022000770,1610612737,ATL,Atlanta,1630168,Onyeka Okongwu,Onyeka,,,19.000000:41,0.306,0.389,0.095,0.160,0.457,0.571,0.210,0.526
21,G0022000770,1610612737,ATL,Atlanta,1629164,Brandon Goodwin,Brandon,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333
22,G0022000770,1610612737,ATL,Atlanta,1630219,Skylar Mays,Skylar,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333
23,G0022000770,1610612737,ATL,Atlanta,1630233,Nathan Knight,Nathan,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333


In [207]:
bs.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN',
       'EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT', 'OPP_EFG_PCT',
       'OPP_FTA_RATE', 'OPP_TOV_PCT', 'OPP_OREB_PCT'],
      dtype='object')

In [208]:
bs2 = bs.merge(gids[['GAME_ID','SEASON_YEAR']],how='left',on='GAME_ID')
bs2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,SEASON_YEAR
0,G0021900502,1610612744,GSW,Golden State,203922,Glenn Robinson III,Glenn,F,,39.000000:33,0.581,0.206,0.117,0.171,0.554,0.176,0.076,0.206,2019-20
1,G0021900502,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,F,,35.000000:48,0.590,0.180,0.133,0.212,0.546,0.169,0.096,0.138,2019-20
2,G0021900502,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,C,,33.000000:57,0.552,0.207,0.111,0.219,0.532,0.113,0.091,0.172,2019-20
3,G0021900502,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,G,,39.000000:33,0.536,0.214,0.115,0.179,0.521,0.181,0.065,0.176,2019-20
4,G0021900502,1610612744,GSW,Golden State,202692,Alec Burks,Alec,G,,38.000000:14,0.551,0.191,0.084,0.222,0.544,0.162,0.105,0.161,2019-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,G0022000770,1610612737,ATL,Atlanta,1630168,Onyeka Okongwu,Onyeka,,,19.000000:41,0.306,0.389,0.095,0.160,0.457,0.571,0.210,0.526,2020-21
379,G0022000770,1610612737,ATL,Atlanta,1629164,Brandon Goodwin,Brandon,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21
380,G0022000770,1610612737,ATL,Atlanta,1630219,Skylar Mays,Skylar,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21
381,G0022000770,1610612737,ATL,Atlanta,1630233,Nathan Knight,Nathan,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21


In [209]:
bs2['YEAR'] = [i[:4] for i in bs2['SEASON_YEAR']]
bs2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,SEASON_YEAR,YEAR
0,G0021900502,1610612744,GSW,Golden State,203922,Glenn Robinson III,Glenn,F,,39.000000:33,0.581,0.206,0.117,0.171,0.554,0.176,0.076,0.206,2019-20,2019
1,G0021900502,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,F,,35.000000:48,0.590,0.180,0.133,0.212,0.546,0.169,0.096,0.138,2019-20,2019
2,G0021900502,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,C,,33.000000:57,0.552,0.207,0.111,0.219,0.532,0.113,0.091,0.172,2019-20,2019
3,G0021900502,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,G,,39.000000:33,0.536,0.214,0.115,0.179,0.521,0.181,0.065,0.176,2019-20,2019
4,G0021900502,1610612744,GSW,Golden State,202692,Alec Burks,Alec,G,,38.000000:14,0.551,0.191,0.084,0.222,0.544,0.162,0.105,0.161,2019-20,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,G0022000770,1610612737,ATL,Atlanta,1630168,Onyeka Okongwu,Onyeka,,,19.000000:41,0.306,0.389,0.095,0.160,0.457,0.571,0.210,0.526,2020-21,2020
379,G0022000770,1610612737,ATL,Atlanta,1629164,Brandon Goodwin,Brandon,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020
380,G0022000770,1610612737,ATL,Atlanta,1630219,Skylar Mays,Skylar,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020
381,G0022000770,1610612737,ATL,Atlanta,1630233,Nathan Knight,Nathan,,,3.000000:36,0.200,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020


In [210]:
bs2['PLAYER_TEAM_SEASON'] = bs2['PLAYER_ID'].astype(str)+'_'+bs2['TEAM_ID'].astype(str)+'_'+bs2['YEAR']
bs2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,SEASON_YEAR,YEAR,PLAYER_TEAM_SEASON
0,G0021900502,1610612744,GSW,Golden State,203922,Glenn Robinson III,Glenn,F,,39.000000:33,...,0.206,0.117,0.171,0.554,0.176,0.076,0.206,2019-20,2019,203922_1610612744_2019
1,G0021900502,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,F,,35.000000:48,...,0.180,0.133,0.212,0.546,0.169,0.096,0.138,2019-20,2019,203110_1610612744_2019
2,G0021900502,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,C,,33.000000:57,...,0.207,0.111,0.219,0.532,0.113,0.091,0.172,2019-20,2019,1627737_1610612744_2019
3,G0021900502,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,G,,39.000000:33,...,0.214,0.115,0.179,0.521,0.181,0.065,0.176,2019-20,2019,1627814_1610612744_2019
4,G0021900502,1610612744,GSW,Golden State,202692,Alec Burks,Alec,G,,38.000000:14,...,0.191,0.084,0.222,0.544,0.162,0.105,0.161,2019-20,2019,202692_1610612744_2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,G0022000770,1610612737,ATL,Atlanta,1630168,Onyeka Okongwu,Onyeka,,,19.000000:41,...,0.389,0.095,0.160,0.457,0.571,0.210,0.526,2020-21,2020,1630168_1610612737_2020
379,G0022000770,1610612737,ATL,Atlanta,1629164,Brandon Goodwin,Brandon,,,3.000000:36,...,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1629164_1610612737_2020
380,G0022000770,1610612737,ATL,Atlanta,1630219,Skylar Mays,Skylar,,,3.000000:36,...,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1630219_1610612737_2020
381,G0022000770,1610612737,ATL,Atlanta,1630233,Nathan Knight,Nathan,,,3.000000:36,...,0.800,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1630233_1610612737_2020


In [211]:
bs2['ngroup'] = bs2.groupby('PLAYER_TEAM_SEASON').ngroup()

In [212]:
bs2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,OREB_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,SEASON_YEAR,YEAR,PLAYER_TEAM_SEASON,ngroup
0,G0021900502,1610612744,GSW,Golden State,203922,Glenn Robinson III,Glenn,F,,39.000000:33,...,0.117,0.171,0.554,0.176,0.076,0.206,2019-20,2019,203922_1610612744_2019,235
1,G0021900502,1610612744,GSW,Golden State,203110,Draymond Green,Draymond,F,,35.000000:48,...,0.133,0.212,0.546,0.169,0.096,0.138,2019-20,2019,203110_1610612744_2019,210
2,G0021900502,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,C,,33.000000:57,...,0.111,0.219,0.532,0.113,0.091,0.172,2019-20,2019,1627737_1610612744_2019,23
3,G0021900502,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,G,,39.000000:33,...,0.115,0.179,0.521,0.181,0.065,0.176,2019-20,2019,1627814_1610612744_2019,37
4,G0021900502,1610612744,GSW,Golden State,202692,Alec Burks,Alec,G,,38.000000:14,...,0.084,0.222,0.544,0.162,0.105,0.161,2019-20,2019,202692_1610612744_2019,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,G0022000770,1610612737,ATL,Atlanta,1630168,Onyeka Okongwu,Onyeka,,,19.000000:41,...,0.095,0.160,0.457,0.571,0.210,0.526,2020-21,2020,1630168_1610612737_2020,150
379,G0022000770,1610612737,ATL,Atlanta,1629164,Brandon Goodwin,Brandon,,,3.000000:36,...,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1629164_1610612737_2020,108
380,G0022000770,1610612737,ATL,Atlanta,1630219,Skylar Mays,Skylar,,,3.000000:36,...,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1630219_1610612737_2020,159
381,G0022000770,1610612737,ATL,Atlanta,1630233,Nathan Knight,Nathan,,,3.000000:36,...,0.148,0.250,0.500,0.800,0.148,0.333,2020-21,2020,1630233_1610612737_2020,163


In [214]:
bs2['ngroup'].value_counts().sort_index()

0      1
1      4
2      1
3      1
4      1
      ..
248    1
249    1
250    1
251    1
252    1
Name: ngroup, Length: 253, dtype: int64

In [215]:
bs2['PLAYER_ID'].value_counts().sort_values()

1629604    1
203894     1
1627767    1
203081     1
202738     1
          ..
1629672    7
1629027    7
1628981    7
1628989    8
1629164    8
Name: PLAYER_ID, Length: 233, dtype: int64

In [216]:
bs2.to_csv('temp.csv')